In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
filename = 'StockData.csv'
plot_on = True
length = 1  # Time period length in years
#GoogDriveDirectory = '/content/drive/My Drive/YF'
PickMarket = 'MYSTOCK'  # Market selection
start_date = '2014-01-01'
end_date = '2024-12-31'
interval = '1d'
period = 'nd'  # 'max' or specific period

# Function to get tickers from a market
def get_tickers(market, filename):
    market_urls = {
        'SP500': 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies',
        'SP100': 'https://en.wikipedia.org/wiki/S%26P_100',
        'DOW30': 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average',
        'NASDAQ100': 'https://en.wikipedia.org/wiki/Nasdaq-100',
        'FTSEMIB': 'https://en.wikipedia.org/wiki/FTSE_MIB',
        'DAX40': 'https://en.wikipedia.org/wiki/DAX',
        'Currencies': 'https://finance.yahoo.com/currencies/'
    }

    # Handle special markets
    if market == 'SP500INDEX':
        return ['^GSPC'], 'SPIndex'
    elif market == 'DJI':
        return ['^DJI'], 'DOWIndex'
    elif market == 'DAX40Index':
        return ['^GDAXI'], 'DAXIndex'
    elif market == 'MYSTOCK':  # Modified to include new stocks
        return ['AAPL', 'AMZN', 'GOOG', 'IBM', 'MSFT', 'NVDA'], 'MyStock'

    # Fetch tickers from Wikipedia
    if market in market_urls:
        url = market_urls[market]
        try:
            tables = pd.read_html(url)
            if market == 'SP500':
                df = tables[0]
            elif market in ['SP100']:
                df = tables[2]
            elif market in ['DOW30', 'NASDAQ100', 'FTSEMIB']:
                df = tables[1]
            elif market == 'DAX40':
                df = tables[4]

            if market in ['FTSEMIB', 'DAX40']:
                df = df.rename(columns={'Ticker': 'Symbol'})

            tickers = df['Symbol'].dropna().tolist()
            print(f"Tickers fetched for {market}: {tickers}")
            return tickers, market
        except Exception as e:
            print(f"Error fetching tickers for {market}: {e}")
            return [], market
    else:
        print(f"Unknown market: {market}")
        return [], filename

# Function to download stock data
def download_stock_data(tickers, start_date, end_date, interval, period):
    all_data = pd.DataFrame()
    download_success = False

    for ticker in tickers:
        try:
            if period == 'max':
                stock_data = yf.download(ticker, period='max', interval=interval)
            else:
                stock_data = yf.download(ticker, start=start_date, end=end_date, interval=interval)

            stock_data['Ticker'] = ticker
            all_data = pd.concat([all_data, stock_data])
            download_success = True
        except Exception as e:
            print(f"Could not download data for {ticker}: {e}")

    if not all_data.empty:
        # Pivot table for closing prices
        prices = all_data.pivot_table(index=all_data.index, columns='Ticker', values='Close')

        # Filter for stocks with complete data
        prices_merge = prices.dropna(axis=1)
        print("Filtered merged prices with full data:")
        print(prices_merge)
        return prices, prices_merge, download_success

    else:
        print("No data downloaded.")
        return pd.DataFrame(), pd.DataFrame(), False


# Main execution
tickers, FileName = get_tickers(PickMarket, filename)

if tickers:
    print(f"Downloading data for {len(tickers)} tickers from the {PickMarket} market...")
    prices, prices_merge, download_success = download_stock_data(tickers, start_date, end_date, interval, period)

    if download_success:
        FilenameClean = f'{FileName}_merge.csv'
        FilenameRaw = f'{FileName}_wholedata.csv'

        # Save merged prices to a local CSV file
        prices_merge.to_csv(FilenameClean, index=True)
        print(f"Merged prices saved locally as {FilenameClean}")

        # Save all prices to a local CSV file
        prices.to_csv(FilenameRaw, index=True)
        print(f"All prices saved locally as {FilenameRaw}")
    else:
        print("No data to save.")

else:
    print(f"No tickers found for market: {PickMarket}")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Filtered merged prices with full data:
Ticker            AAPL        AMZN        GOOG         IBM        MSFT  \
Ticker            AAPL        AMZN        GOOG         IBM        MSFT   
Date                                                                     
2014-01-02   17.215368   19.898500   27.593388  111.759750   30.996416   
2014-01-03   16.837229   19.822001   27.392099  112.428383   30.787872   
2014-01-06   16.929031   19.681499   27.697502  112.042831   30.137259   
2014-01-07   16.807968   19.901501   28.231462  114.277710   30.370813   
2014-01-08   16.914400   20.096001   28.290213  113.229568   29.828627   
...                ...         ...         ...         ...         ...   
2024-12-23  254.989655  225.059998  195.766968  220.461243  434.379028   
2024-12-24  257.916443  229.050003  197.345184  222.924835  438.450836   
2024-12-26  258.735504  227.050003  196.875717  223.401657  437.233276   
2024-12-27  255.309296  223.750000  193.819183  221.305618  429.668457   


C:\Users\Sourav\AppData\Local\Temp\ipykernel_33624\1275707112.py:88: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  prices = all_data.pivot_table(index=all_data.index, columns='Ticker', values='Close')


In [3]:
tickers

['AAPL', 'AMZN', 'GOOG', 'IBM', 'MSFT', 'NVDA']